In [1]:
'''Schritt 3
Die in Schritt 1 und 2 heruntergeladenen Daten werden
zusammengefügt 
'''

In [3]:
import json 
import pandas as pd
import geopandas as gpd

import glob                             #verwendung von 'wildacards' in Dateinamen
from tqdm import tqdm                   #fuer Darstellen eines Fortschrittsbalkens
from datetime import date
import matplotlib.pyplot as plt

#import seaborn as sns 

print('import successful')

import successful


In [4]:
'''Einlesen der Daten 

Erwartet dass unter OBS-roh eine Datei (geojson) liegt, die vorher mit 1-1_get-newest-OBS erstellt wurde
Erwartet dass unter OSM-roh eine Datei (gpkg) liegt, die vorher mit 1-0_OSM-aufbereiten erstellt wurde'''

#def OSM2OBS(path_OSM, path_OBS):        #erwartet Pfad des Unterordners an dem OSM-Datei als *.gpkg liegt und OBS als *.geojson

#Pfade erstellen (für Verwendung in Funktion jeweils erste Zeile weglassen)
path_OSM = '../OSM-roh'
path_OSM = path_OSM + '/*.gpkg'

path_OBS = '../OBS-roh'
path_OBS = path_OBS + '/*BBB.geojson'

#liest alle obs-Dateien ein und packt sie in eine liste. 
files_OBS = glob.glob(path_OBS)

gdf =gpd.GeoDataFrame()
#nimmt alle Dateien (Frames), versieht sie mit einem Zusatz der das Quellportal benennt und packt alles in einen zusammenhängenden Datensatz
for f in tqdm(files_OBS):
    gdf_this = gpd.read_file(f, crs='EPSG:4326')
    gdf = gpd.GeoDataFrame(pd.concat([gdf, gdf_this], ignore_index=True), crs=gdf_this.crs)  # ersetzt: gdf=gdf.append(gdf_this), da append veraltet ist.

#liest alle (sollte aber nur eine sein!) Dateien aus dem Verzeichnis mit osm Dateien (Format gpkg)
files_OSM = glob.glob(path_OSM)


print("OSM: " + str(files_OSM) + "\nOBS: " + str(files_OBS))

100%|██████████| 1/1 [00:01<00:00,  1.51s/it]

OSM: ['../OSM-roh/drivable_infra_Brandenburg_231211.gpkg']
OBS: ['../OBS-roh/231122_OBS-in-BBB.geojson']


In [5]:
#liest die gpkg mit osm daten ein, hier aber nur die linien.
osm_netz=gpd.read_file(files_OSM[0], layer='lines')   

# plot every 100th way
#osm_netz[::100].plot(figsize=(10,10))

In [6]:
osm_netz.groupby(osm_netz['highway']).count()

,osm_id,name,ref,maxspeed,zone_traffic,lanes,lane_markings,lit,surface,bicycle,cycleway,cycleway_both,cycleway_left,cycleway_right,width,tracktype,smoothness,oneway,cycleway_lane,cycleway_both_lane,cycleway_right_lane,cycleway_left_lane,parking_lane_both,parking_lane_left,parking_lane_right,cycleway_separation_left,cycleway_separation_right,separation_left,separation_right,cycleway_bicycle,cycleway_both_bicycle,cycleway_right_bicycle,cycleway_left_bicycle,bicycle_road,cycleway_width,cycleway_both_width,cycleway_right_width,cycleway_left_width,cycleway_right_marking_left,cycleway_right_marking_right,...,parking,parking_both,parking_left,parking_right,cycleway_right_separation_right,cycleway_left_separation_right,cycleway_both_separation_right,parking_both_orientation,parking_left_orientation,parking_right_orientation,cycleway_right_separation_left,cycleway_left_separation_left,cycleway_both_separation_left,cycleway_right_separation_both,cycleway_left_separation_both,cycleway_both_separation_both,cycleway_right_marking_both,cycleway_left_marking_both,cycleway_both_marking_left,cycleway_both_marking_right,cycleway_both_traffic_sign,cycleway_both_marking_both,cycleway_right_buffer_right,cycleway_left_buffer_right,cycleway_both_buffer_right,cycleway_right_buffer_left,cycleway_left_buffer_left,cycleway_both_buffer_left,cycleway_right_buffer_both,cycleway_left_buffer_both,cycleway_both_buffer_both,cycleway_marking_both,cycleway_marking_left,cycleway_marking,cycleway_right_marking,cycleway_left_marking,cycleway_traffic_sign,parking_orientation,other_tags,geometry
highway,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bridleway,20,0,0,0,0,0,0,5,16,20,0,0,0,0,5,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,20
construction,45,25,4,18,0,11,1,30,41,44,0,5,0,2,12,0,14,9,0,0,0,0,4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,45
cycleway,8063,283,12,47,0,78,4,6342,7103,1409,2392,3,0,1,1824,49,3350,6029,1,0,0,0,0,0,0,1,0,307,278,0,0,0,0,28,59,0,0,0,0,0,...,0,8,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5923,8063
elevator,14,5,0,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,14
footway,229275,5671,2,35,2,28,3,130091,151417,19401,67,2,5,0,9252,450,49104,877,0,0,0,0,1,0,1,0,0,4,0,0,0,0,0,5,126,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,107695,229275
living_street,6578,5858,0,507,7,403,81,3002,4643,222,64,1090,1,1,629,32,1630,762,0,0,0,0,55,46,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,591,203,198,0,0,0,91,182,185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4220,6578
motorway_link,2168,44,49,1253,14,2150,19,1534,1601,31,4,0,0,0,7,0,58,2168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1903,2168
path,64416,2502,30,63,1,21,2,11067,40050,21436,138,2,1,1,9960,2015,13523,2966,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,49,191,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,33010,64416
pedestrian,975,604,0,36,0,9,0,639,813,337,2,54,4,4,32,4,318,27,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,...,0,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,699,975


In [7]:
#dropna: entfernt alle Datensätze mit NA in entsprechender Spalte
gdf=gdf.dropna(subset=['way_id']).copy()
len(gdf)

21046

In [8]:
#sicherstellen, dass die ID in beiden Datensätzen (OSM und OBS) vom selben Typ (Integer) ist
gdf['way_id']=gdf['way_id'].astype(int)
osm_netz['osm_id']=osm_netz['osm_id'].astype(int)
len(gdf)

21046

In [9]:
# erzeugt einen neuen GeoDataFrame und fügt darin die Spalten aus dem OSM-Elementen den OBS-Elementen mit der identischen ID hinzu.
gdf_merge=gpd.GeoDataFrame()
gdf_merge=gdf.merge(osm_netz, left_on='way_id', right_on='osm_id')  # , how="left"  : Hinzufügen, dann bleiben OBS-Werte enthalten, auch wenn es keine Entsprechung in OSM gibt. Sind bei mir auch Messungen außerhalb Brandenburg!!
len(gdf_merge)

19706

In [12]:
gdf_kombi = gdf_merge

datum=date.today().strftime("%y%m%d")

gdf_kombi = gpd.GeoDataFrame(gdf_kombi, geometry='geometry_x', crs='EPSG:4326')
gdf_kombi = gdf_kombi.astype({'geometry_y':'string'})

gdf_kombi.to_file("../OSM-OBS_merged/OSM-OBS_merge"+datum+".gpkg", driver="GPKG")
#gdf_kombi.to_file("../OSM-OBS_merged/OSM-OBS_merge230315.gpkg", driver="GPKG")
#gdf.to_file("../OSM-OBS_merged/OSM-OBS_merge"+datum+".gpkg", driver="GPKG")